# WatchTower
In this notebook we will pull entities from watch tower.

In [10]:
# install all the azure identity modules
%pip install azure.identity
# install all the azure keyvault modules
%pip install azure-keyvault-secrets
# install pandas modules
%pip install pandas
# install all the azure sql database modules
%pip install pyodbc

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\huanminw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\huanminw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\huanminw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\huanminw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
from azure.identity import AzureCliCredential, ChainedTokenCredential, DefaultAzureCredential, ManagedIdentityCredential
from azure.keyvault.secrets import SecretClient
from azure.identity import ClientSecretCredential, CertificateCredential
import struct
import pyodbc
import pandas as pd 
import requests;

class DatabaseConnection():
    def __init__(self, configuration : dict):
        server = configuration.get('server')
        if (not str(server).endswith('.database.windows.net')) :
            server = server + '.database.windows.net';
        self.sever = server;
        self.database = configuration.get('database');
        credential = ChainedTokenCredential(ManagedIdentityCredential(), 
                                            AzureCliCredential(), 
                                            DefaultAzureCredential());
        databaseToken = credential.get_token('https://database.windows.net/');
        print(databaseToken);
        tokenb = bytes(databaseToken[0], "UTF-8");
        exptoken = b'';
        for i in tokenb:
            exptoken += bytes({i});
            exptoken += bytes(1);
        tokenstruct = struct.pack("=i", len(exptoken)) + exptoken;
        connString = "Driver={ODBC Driver 17 for SQL Server};SERVER="+self.sever+";DATABASE="+self.database+"";
        SQL_COPT_SS_ACCESS_TOKEN = 1256; 
        self.connection = pyodbc.connect(connString, attrs_before = {SQL_COPT_SS_ACCESS_TOKEN:tokenstruct});


    # write a database to database, mode = append or replace
    def insert(self, df : pd.DataFrame, table :str):
        """
        Execute a SQL write.
        @param df: Data Frame.        
        @param table: Table Name.
        """
        statement = '''
        INSERT INTO {table}
        (
            {columns}
        )
        VALUES({place_holders})
        '''
        # add protection on column name
        columns = df.columns.tolist();
        for i in range(len(columns)):
            columns[i] = '[' + columns[i] + ']';
        statement = statement.format(table = table, 
                                     columns = (',').join(columns), 
                                     place_holders = (',').join(['?'] * len(df.columns)));
        print(statement);
        # Create a Pandas dataframe from the results
        cursor = self.connection.cursor();
        cursor.fast_executemany = True;
        cursor.executemany(statement, df.values.tolist());
        cursor.commit();
        cursor.close();

    # execute sql command
    def execute(self, statement : str):
        """
        Execute a SQL command.
        @param query: query.        
        """
        # Create a Pandas dataframe from the results
        cursor = self.connection.cursor();
        cursor.execute(statement);
        cursor.commit();
        cursor.close();


    def query(self, query: str) -> pd.DataFrame:
        """
        Execute a SQL query.
        @param query: The query.
        @returns: A Pandas DataFrame.
        """
        # Create a Pandas dataframe from the results
        
        df = pd.read_sql(query, self.connection);

        # Print the dataframe
        print(df);
        return df

    def close(self):
        """
        Close the Database client.
        """
        self.connection.close();

class KeyVault():
    def __init__(self, configuration : dict):
        key_vault = configuration.get('keyvault');
        credential = ChainedTokenCredential(ManagedIdentityCredential(), 
                                            AzureCliCredential(), 
                                            DefaultAzureCredential());
        kv_url = "https://{key_vault}.vault.azure.net";
        kv_url = kv_url.format(key_vault = key_vault);
        self.client = SecretClient(vault_url=kv_url, credential=credential)

    # write a database to database, mode = append or replace
    def get(self, secret_name : str) -> str:
        secret_value = self.client.get_secret(secret_name);
        return secret_value.value;


class RestAPI():
    def __init__(self, configuration : dict):
        app_id = configuration.get('app_id');
        app_secret = configuration.get('app_secret');
        tenant_id = '72f988bf-86f1-41af-91ab-2d7cd011db47';
        credential = ClientSecretCredential(tenant_id, app_id, app_secret);
        #credential = CertificateCredential(tenant_id, app_id, certificate_path ='CN=capacityinsightprd00.geneva.keyvault.com');
        self.token = credential.get_token("280b00f4-2c5a-44d8-bab9-6acbbaaa7c78/.default").token;
    
    def call_data_api(self, uri : str) -> dict:
        result = [];
        try:
            headers = {
                "Authorization": "Bearer " + self.token,
                "Content-Type": "application/json"  # Adjust content type as per your API
            }
            response = requests.get(uri, headers=headers);
            if response.status_code == 200:
                result = response.json();
            else:
                print("Error:", response.status_code)
        except requests.exceptions.RequestException as e:
            print("Error:", e);
        return result;

key_vault = KeyVault({'keyvault' : 'COICDIKV'});
app_id = key_vault.get('serviceprincipal');
app_secret = key_vault.get('serviceprincipalkey');
app_cert = key_vault.get('capacityinsightprd00');
print(app_cert);
api_client = RestAPI({'app_id' : app_id, 'app_secret': app_secret, 'app_cert': app_cert});
result = api_client.call_data_api("https://api.infra.global.azure.com/pantheon/api/v2/entities/BuildoutProjectEntity")
projects = result['result'];
print(projects)
# Create an empty DataFrame   
table = [];
for p in projects:
    record = [];
    record.append(p['projectOid']);
    record.append(p['title']);    
    record.append(p['maLaunchDate']);
    record.append(p['maLaunchState']);
    record.append(p['uaLaunchDate']);
    record.append(p['uaLaunchState']); 
    record.append(p['earlyAccessDate']);
    record.append(p['earlyAccessStatus']);    
    record.append(p['buildoutStartDate']);
    record.append(p['buildoutStartStatus']);    
    record.append(p['regionOid']);
    record.append(p['projectStatus']);
    table.append(record);
df = pd.DataFrame(table, columns=['projectOid', 'title', 'maLaunchDate', 'maLaunchState', 'uaLaunchDate', 'uaLaunchState', 'earlyAccessDate', 
                                  'earlyAccessStatus', 'buildoutStartDate', 'buildoutStartStatus', 'regionOid', 'projectStatus']);
#                            'businessScenario', 'arid', 'regionScorecardId', 'buildoutPSL', 'changedOn', 'changedBy', \
#                            , 'projectSupplyPlanDate', 'projectDCReadyDate', 'projectRNIDate', \
#                            'projectShellReadyDate', 'projectRFSDate', 'projectCOLOReadyAchieved', 'projectDCReadyAchieved', 'projectRNIAchieved', \
#                            'projectShellReadyAchieved', 'projectDockDate', 'projectDockState', 'projectRTEGDate', 'projectRTEGState', \
#                            'projectRTWDate', 'projectRTWState', 'projectLiveDate', 'projectLiveState', 'itCapacityMW', 'publicDisclosureDate', \
#                            'internalDisclosureDate', 'projectCOLODateManualOverride', 'projectNeedByDate', 'createdBy', 'createdOn', 'updatedBy', \
#                            'updatedOn', 'projectArchitecture', 'ring0Services', 'rbSv2State', 'zoneEnablement', 'projectRFSAchieved', \
#                            'regionDesignId', 'businessScenarioValue', 'projectStatusValue', 'targetRegionType', 'completedOn', 'opportunityType', \
#                            'projectOwnerOid', 'buildoutStartDate', 'buildoutStartStatus', 'afsCompleteDate', 'afsCompleteStatus', 'amsCompleteDate', \
#                            'amsCompleteStatus', 'coloReadyDate', 'coloReadyStatus', 'entityName', 'id', '_etag']);

#df = pd.DataFrame(table, columns=[ 'isDeleted', 'projectOid', 'projectId', 'title', 'buildoutPM', 'regionOid', 'projectStatus', 'projectState', \
#                            'businessScenario', 'arid', 'regionScorecardId', 'buildoutPSL', 'changedOn', 'changedBy', 'maLaunchDate', \
#                            'maLaunchState', 'uaLaunchDate', 'uaLaunchState', 'projectSupplyPlanDate', 'projectDCReadyDate', 'projectRNIDate', \
#                            'projectShellReadyDate', 'projectRFSDate', 'projectCOLOReadyAchieved', 'projectDCReadyAchieved', 'projectRNIAchieved', \
#                            'projectShellReadyAchieved', 'projectDockDate', 'projectDockState', 'projectRTEGDate', 'projectRTEGState', \
#                            'projectRTWDate', 'projectRTWState', 'projectLiveDate', 'projectLiveState', 'itCapacityMW', 'publicDisclosureDate', \
#                            'internalDisclosureDate', 'projectCOLODateManualOverride', 'projectNeedByDate', 'createdBy', 'createdOn', 'updatedBy', \
#                            'updatedOn', 'projectArchitecture', 'ring0Services', 'rbSv2State', 'zoneEnablement', 'projectRFSAchieved', \
#                            'regionDesignId', 'businessScenarioValue', 'projectStatusValue', 'targetRegionType', 'completedOn', 'opportunityType', \
#                            'projectOwnerOid', 'buildoutStartDate', 'buildoutStartStatus', 'afsCompleteDate', 'afsCompleteStatus', 'amsCompleteDate', \
#                            'amsCompleteStatus', 'coloReadyDate', 'coloReadyStatus', 'earlyAccessDate', 'earlyAccessStatus', 'entityName', 'id', '_etag']);

#databaseConn = DatabaseConnection({'server' : 'capacityreporting.database.windows.net', 'database' :'bireporting'});
#databaseConn.execute('DELETE FROM [STG_WatchTower].[BuildoutProjectEntity]')
#databaseConn.insert(df, table = '[STG_WatchTower].[BuildoutProjectEntity]');
#databaseConn.execute('EXEC [WatchTower].[ProcessBuildProjectEntity]')
#databaseConn.execute('EXEC [WatchTower].[ProcessProjectColoTranches]')
#databaseConn.close();
print(df);


HttpResponseError: (Forbidden) The user, group or application 'appid=04b07795-8ddb-461a-bbee-02f9e1bf7b46;oid=92ec815f-dc4c-4eff-af4f-34c279db8bb3;numgroups=1184;iss=https://sts.windows.net/72f988bf-86f1-41af-91ab-2d7cd011db47/' does not have secrets get permission on key vault 'COICDIKV;location=eastus2'. For help resolving this issue, please see https://go.microsoft.com/fwlink/?linkid=2125287
Code: Forbidden
Message: The user, group or application 'appid=04b07795-8ddb-461a-bbee-02f9e1bf7b46;oid=92ec815f-dc4c-4eff-af4f-34c279db8bb3;numgroups=1184;iss=https://sts.windows.net/72f988bf-86f1-41af-91ab-2d7cd011db47/' does not have secrets get permission on key vault 'COICDIKV;location=eastus2'. For help resolving this issue, please see https://go.microsoft.com/fwlink/?linkid=2125287
Inner error: {
    "code": "AccessDenied"
}